<a href="https://colab.research.google.com/github/NaNa0369/SA_2025/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import bokeh
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
from bokeh.layouts import column

# Setting up Bokeh for notebook output
output_notebook()

# Loading the dataset
def load_parking_data(file_path="/content/dataset.csv"):
    df = pd.read_csv(file_path)
    # Converting date and time to datetime
    df['Datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                   format='%d-%m-%Y %H:%M:%S')
    return df.sort_values('Datetime')

# Calculating distance between parking lots using Haversine formula
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371e3  # Earth's radius in meters
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Model 1: Baseline Linear Model
def baseline_linear_model(occupancy, capacity, previous_price, alpha=0.5):
    occupancy_rate = occupancy / capacity
    price_adjustment = alpha * occupancy_rate
    return previous_price + price_adjustment

# Model 2: Demand-Based Price Function
def demand_based_model(row, base_price=10.0):
    # Parameters
    alpha, beta, gamma, delta, epsilon = 0.5, 0.1, 0.05, 0.2, 0.1

    # Encoding categorical variables
    traffic_weights = {'low': 0.0, 'average': 0.5, 'high': 1.0}
    vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5, 'cycle': 0.3}

    occupancy_rate = row['Occupancy'] / row['Capacity']
    traffic = traffic_weights.get(row['TrafficConditionNearby'], 0.5)
    vehicle_weight = vehicle_weights.get(row['VehicleType'], 1.0)

    # Demand function
    demand = (alpha * occupancy_rate +
              beta * row['QueueLength'] -
              gamma * traffic +
              delta * row['IsSpecialDay'] +
              epsilon * vehicle_weight)

    # Normalizing demand to [0, 1]
    normalized_demand = np.clip(demand, 0, 1)

    # Price calculation with smoothing
    lambda_factor = 0.8
    new_price = base_price * (1 + lambda_factor * normalized_demand)

    # Bounding price between 0.5x and 2x base price
    new_price = np.clip(new_price, base_price * 0.5, base_price * 2.0)

    return new_price

# Model 3: Competitive Pricing Model
def competitive_pricing_model(row, df, current_index, base_price=10.0):
    current_price = demand_based_model(row, base_price)

    # Finding nearby lots at the same timestamp
    current_time = row['Datetime']
    current_lat, current_lon = row['Latitude'], row['Longitude']
    system_code = row['SystemCodeNumber']

    # Getting competitor data at the same or closest previous timestamp
    competitor_data = df[(df['Datetime'] <= current_time) &
                       (df['SystemCodeNumber'] != system_code)]
    competitor_data = competitor_data.groupby('SystemCodeNumber').last().reset_index()

    competitor_prices = []
    for _, comp_row in competitor_data.iterrows():
        distance = haversine_distance(
            current_lat, current_lon,
            comp_row['Latitude'], comp_row['Longitude']
        )
        if distance < 1000:  # Considering lots within 1km
            comp_price = demand_based_model(comp_row, base_price)
            competitor_prices.append(comp_price)

    # Adjusting price based on competitors
    reroute_suggestion = None
    if competitor_prices:
        avg_comp_price = np.mean(competitor_prices)
        if current_price > avg_comp_price * 1.2 and row['Occupancy'] / row['Capacity'] > 0.9:
            # If price is too high and lot is full, suggest rerouting
            current_price *= 0.9
            reroute_suggestion = "Suggest rerouting to nearby lots"
        elif avg_comp_price > current_price:
            # If competitors are more expensive, increase price slightly
            current_price = min(current_price * 1.1, base_price * 2.0)

    return current_price, reroute_suggestion

# Processing data and generating prices
def process_data(df):
    results = []
    prev_prices = {}  # Tracking previous prices per parking lot
    base_price = 10.0

    for index, row in df.iterrows():
        system_code = row['SystemCodeNumber']
        prev_price = prev_prices.get(system_code, base_price)

        # Model 1: Baseline Linear
        price_linear = baseline_linear_model(row['Occupancy'], row['Capacity'], prev_price)

        # Model 2: Demand-Based
        price_demand = demand_based_model(row)

        # Model 3: Competitive Pricing
        price_competitive, reroute_suggestion = competitive_pricing_model(row, df, index)

        # Updating previous price
        prev_prices[system_code] = price_competitive

        results.append({
            'Datetime': row['Datetime'],
            'SystemCodeNumber': system_code,
            'Price_Linear': price_linear,
            'Price_Demand': price_demand,
            'Price_Competitive': price_competitive,
            'Reroute_Suggestion': reroute_suggestion,
            'Occupancy_Rate': row['Occupancy'] / row['Capacity']
        })

    return pd.DataFrame(results)

# Creating Bokeh visualization
def create_bokeh_plots(df_results):
    source = ColumnDataSource(df_results)

    p = figure(title="Real-time Parking Prices", x_axis_type="datetime",
               height=400, width=800)
    p.line(x='Datetime', y='Price_Linear', source=source, legend_label="Linear Model",
           line_color="blue")
    p.line(x='Datetime', y='Price_Demand', source=source, legend_label="Demand Model",
           line_color="green")
    p.line(x='Datetime', y='Price_Competitive', source=source,
           legend_label="Competitive Model", line_color="red")
    p.line(x='Datetime', y='Occupancy_Rate', source=source, legend_label="Occupancy Rate",
           line_color="purple", line_dash="dashed")

    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    p.legend.click_policy = "hide"

    return p

# Main execution
def main():
    # Loading data
    df = load_parking_data()

    # Processing data
    df_results = process_data(df)

    # Creating and displaying visualization
    plot = create_bokeh_plots(df_results)
    show(plot)

    # Saving results to CSV for reference
    df_results.to_csv('parking_pricing_results.csv', index=False)

    return df_results

# Running the main function
if __name__ == "__main__":
    df_results = main()

# Documentation of Approach
"""
# Dynamic Parking Pricing Model

## Demand Function
The demand function for Model 2 is defined as:
Demand = α * (Occupancy/Capacity) + β * QueueLength - γ * Traffic + δ * IsSpecialDay + ε * VehicleTypeWeight

Where:
- α = 0.5 (weight for occupancy rate)
- β = 0.1 (weight for queue length)
- γ = 0.05 (weight for traffic conditions)
- δ = 0.2 (weight for special day)
- ε = 0.1 (weight for vehicle type)

## Assumptions
1. Base price is $10 for all parking lots.
2. Traffic conditions are weighted: low=0, average=0.5, high=1.0.
3. Vehicle types have different weights: car=1.0, bike=0.5, truck=1.5, cycle=0.3.
4. Prices are bounded between 0.5x and 2x of base price.
5. Competitive model considers lots within 1km radius using Haversine distance.
6. For competitive pricing, competitor data uses the most recent data before or at the current timestamp.

## Price Adjustment Logic
- **Model 1 (Baseline Linear)**: Adjusts price linearly based on occupancy rate, using the previous price for continuity.
- **Model 2 (Demand-Based)**: Uses a demand function incorporating occupancy, queue length, traffic, special day, and vehicle type. Demand is normalized and prices are smoothed with bounds.
- **Model 3 (Competitive)**: Builds on Model 2, adjusting prices based on nearby lots (within 1km). Reduces price and suggests rerouting if the lot is over 90% full and pricier than competitors; increases price if competitors are more expensive.

## Visualizations
- Line plots showing prices from all three models over time for each parking lot.
- Occupancy rate overlaid as a dashed line for reference.
- Interactive legend to toggle lines on/off.
- Plots are generated using Bokeh for real-time visualization effect.

## Output
- Results are saved to 'parking_pricing_results.csv' for further analysis.
- Visualizations are displayed in the notebook and can be interacted with.
"""


"\n# Dynamic Parking Pricing Model\n\n## Demand Function\nThe demand function for Model 2 is defined as:\nDemand = α * (Occupancy/Capacity) + β * QueueLength - γ * Traffic + δ * IsSpecialDay + ε * VehicleTypeWeight\n\nWhere:\n- α = 0.5 (weight for occupancy rate)\n- β = 0.1 (weight for queue length)\n- γ = 0.05 (weight for traffic conditions)\n- δ = 0.2 (weight for special day)\n- ε = 0.1 (weight for vehicle type)\n\n## Assumptions\n1. Base price is $10 for all parking lots.\n2. Traffic conditions are weighted: low=0, average=0.5, high=1.0.\n3. Vehicle types have different weights: car=1.0, bike=0.5, truck=1.5, cycle=0.3.\n4. Prices are bounded between 0.5x and 2x of base price.\n5. Competitive model considers lots within 1km radius using Haversine distance.\n6. For competitive pricing, competitor data uses the most recent data before or at the current timestamp.\n\n## Price Adjustment Logic\n- **Model 1 (Baseline Linear)**: Adjusts price linearly based on occupancy rate, using the pre